In [33]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy 
import pickle
from glob import glob
from torch.autograd import Variable
from statistics import mean

"""Change to the data folder"""
train_path = "./new_train/"
val_path = "./new_val_in/"

# number of sequences in each dataset
# train:205942  val:3200 test: 36272 
# sequences sampled at 10HZ rate

In [3]:
class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.transform = transform

        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):

        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            
        if self.transform:
            data = self.transform(data)

        return data


# intialize a dataset
train_dataset  = ArgoverseDataset(data_path=train_path)
val_dataset =  ArgoverseDataset(data_path=val_path)

In [4]:
batch_sz = 32

def my_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = [numpy.dstack([scene['p_in'], scene['v_in']]) for scene in batch]
    out = [numpy.dstack([scene['p_out'], scene['v_out']]) for scene in batch]
    #a_id =[numpy.dstack([scene['agent_id']]) for scene in batch]
    #t_id =[numpy.dstack([scene['track_id']]) for scene in batch]
    #inp = torch.LongTensor(inp)
    #out = torch.LongTensor(out)
    inp =  torch.FloatTensor(inp)
    out =  torch.FloatTensor(out)
    
    return [inp, out]

In [5]:
batch_sz = 32

def my_collate2(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = [numpy.dstack([scene['p_in'], scene['v_in']]) for scene in batch]
    #out = [numpy.dstack([scene['p_out'], scene['v_out']]) for scene in batch]
    a_id =[numpy.dstack([scene['agent_id']]) for scene in batch]
    t_id =[numpy.dstack([scene['track_id']]) for scene in batch]
    s_id =[numpy.dstack([scene['scene_idx']]) for scene in batch] 
    #inp = torch.LongTensor(inp)
    #out = torch.LongTensor(out)
    inp =  torch.FloatTensor(inp)
    #out =  torch.FloatTensor(out)
    
    return [inp,a_id,t_id,s_id]

In [ ]:
train_loader = DataLoader(train_dataset,batch_size=batch_sz, shuffle = True, collate_fn=my_collate, num_workers=1)
val_loader = DataLoader(val_dataset,batch_size=batch_sz, shuffle = False, collate_fn=my_collate2, num_workers=1)

In [12]:
v1 = train_dataset.__getitem__(0)['v_in']


(60, 19, 2)

In [48]:
scene = train_dataset.__getitem__(0)
scene['p_in'][0][-1]

array([3277.29638672, 1947.62585449])

In [30]:
def find_agent(scene):
    agent_id = scene['agent_id']
    for i in range(60):
        track_id = scene['track_id'][i][0]
        if(agent_id == track_id):
            return i

In [62]:
def avg_agent_velocity(scene):
    v_all = scene['v_in']
    agent_idx = find_agent(scene)
    v_agent = v_all[agent_idx]
    vx = mean(v_agent[:, 0][14:18])
    vy = mean(v_agent[:, 1][14:18])
    return vx, vy

In [63]:
avg_agent_velocity(train_dataset.__getitem__(0))

(3.396494448184967, 2.59128400683403)

In [64]:
vx_all = []
vy_all = []
for i in range(val_dataset.__len__()):
    scene = val_dataset.__getitem__(i)
    vx, vy =avg_agent_velocity(scene)
    vx_all.append(vx)
    vy_all.append(vy)

In [65]:
len(vx_all)

3200

In [66]:
output = []
for i in range(val_dataset.__len__()):
    agent_output = []
    scene = val_dataset.__getitem__(i)
    agent_idx = find_agent(scene)
    start_px = val_dataset.__getitem__(i)['p_in'][agent_idx][-1][0]
    start_py = val_dataset.__getitem__(i)['p_in'][agent_idx][-1][1]
    vx, vy = avg_agent_velocity(scene)
    for j in range(1,31):
        px = start_px  + vx*0.1*j
        py = start_py + vy*0.1*j
        agent_output.append(px)
        agent_output.append(py)
    output.append(agent_output)

In [67]:
# y = ["v1", "v2", "v3", "v4", "v5", "v6", "v7", "v8", "v9", "v10", "v11", "v12", "v13", "v14", "v15", "v16", "v17", "v18", "v19", "v20", "v21", "v22", "v23", "v24", "v25", "v26", "v27", "v28", "v29", "v30", "v31", "v32", "v33", "v34", "v35", "v36", "v37", "v38", "v39", "v40", "v41", "v42", "v43", "v44", "v45", "v46", "v47", "v48", "v49", "v50", "v51", "v52", "v53", "v54", "v55", "v56", "v57", "v58", "v59", "v60"]
output = numpy.array(output)
# var = numpy.stack((y, final))
numpy.savetxt("submission_avg3.csv", output, delimiter=",")
